[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.leafmap.org/lab/index.html?path=notebooks/115_terrascope.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/leafmap/blob/master/docs/notebooks/115_terrascope.ipynb)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeos/leafmap/HEAD)

# Terrascope STAC API with leafmap

This notebook demonstrates how to access Terrascope data using the `leafmap.terrascope` module.

**Setup:** Set environment variables before running:
```bash
export TERRASCOPE_USERNAME='your_username'
export TERRASCOPE_PASSWORD='your_password'
```

In [ ]:
import leafmap
import leafmap.terrascope as terrascope

## Authentication

In [ ]:
terrascope.login()

## Search for NDVI Data

In [ ]:
bbox = [5.032597, 51.220809, 5.055170, 51.234246]

items = terrascope.search_ndvi(
    bbox=bbox,
    start="2025-05-01",
    end="2025-06-01",
    max_cloud_cover=10,
)

print(f"Found {len(items)} scenes:")
for date in terrascope.get_item_dates(items):
    print(f"  {date}")

## Single Layer Visualization

In [ ]:
center = [(bbox[1] + bbox[3]) / 2, (bbox[0] + bbox[2]) / 2]

m = leafmap.Map(center=center, zoom=14)
m.add_raster(
    items[0].assets["NDVI"].href,
    layer_name=f"NDVI {items[0].datetime.date()}",
    colormap="RdYlGn",
    vmin=0,
    vmax=250,
)
m

## Split Map Comparison

In [ ]:
if len(items) >= 2:
    first, last = items[0], items[-1]
    print(f"Comparing: {first.datetime.date()} vs {last.datetime.date()}")

    m = leafmap.Map(center=center, zoom=14)
    m.split_map(
        left_layer=first.assets["NDVI"].href,
        right_layer=last.assets["NDVI"].href,
        left_label=f"NDVI {first.datetime.date()}",
        right_label=f"NDVI {last.datetime.date()}",
    )
    display(m)

## Time Slider Animation

In [ ]:
layers = terrascope.create_time_layers(items[:3])  # Limit to 3 for demo

m = leafmap.Map(center=center, zoom=14)
m.add_time_slider(layers, time_interval=1)
m

## Data Analysis with rioxarray

In [ ]:
import rioxarray
import numpy as np

# Clean up stale tile servers before analysis
terrascope.cleanup_tile_servers()

first_item = items[0]
print(f"Analyzing: {first_item.datetime.date()}")

with rioxarray.open_rasterio(first_item.assets["NDVI"].href, mask_and_scale=True) as ds:
    clipped = ds.rio.clip_box(*bbox, crs="EPSG:4326")
    data = clipped.sel(band=1).values

    print(f"\nNDVI Statistics:")
    print(f"  Min: {np.nanmin(data):.2f}")
    print(f"  Max: {np.nanmax(data):.2f}")
    print(f"  Mean: {np.nanmean(data):.2f}")

## Explore Available Collections

In [ ]:
collections = terrascope.list_collections()
print(f"Available collections ({len(collections)}):")
for c in sorted(collections):
    print(f"  {c}")

In [ ]:
# Optional: logout when done
# terrascope.logout()